In [1]:
from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [2]:
model = CompartmentalModel(
    times=[1850.0, 2024.0],
    compartments=["susceptible", "early_latent", "late_latent", "infectious", "recovered"],
    infectious_compartments=["infectious"],
)
model.set_initial_population({"susceptible": 1.0})

In [3]:
latency_flows = [
    ("stabilisation", "early_latent", "late_latent"),
    ("early_activation", "early_latent", "infectious"),
    ("late_activation", "late_latent", "infectious"),
]
for flow, source, dest in latency_flows:
    model.add_transition_flow(flow, Parameter(f"{flow}_rate"), source, dest)

In [4]:
infection_origins = ["susceptible", "late_latent", "recovered"]
for origin in infection_origins:
    model.add_infection_frequency_flow(f"infection_from_{origin}", Parameter("contact_rate"), origin, "early_latent")

In [5]:
params = {
    "contact_rate": 1.0,
    "stabilisation_rate": 1.0,
    "early_activation_rate": 1.0,
    "late_activation_rate": 1.0,
}

In [10]:
from IPython.display import Markdown

In [13]:
import pandas as pd

Markdown(pd.Series(params).to_markdown())

|                       |   0 |
|:----------------------|----:|
| contact_rate          |   1 |
| stabilisation_rate    |   1 |
| early_activation_rate |   1 |
| late_activation_rate  |   1 |

In [6]:
model.run(params)

In [14]:
# model.flows